In [ ]:
from google.colab import drive
drive.mount('/content/drive')


import os

import torch
os.chdir('/content/drive/MyDrive/ImitationLearning/Invariant-Causal-Imitation-Learning-main/')


Mounted at /content/drive


# load

In [ ]:
!pip install mpi4py 
!pip install box2d-py
!pip install box2d 
!pip3 install gym[Box_2D] 
!pip install gym==0.17.2 -qqq
!pip install numpy~=1.18.2 -qqq
!pip install pandas~=1.0.4 -qqq
!pip install PyYAML~=5.4.1 -qqq
!pip install scikit-learn~=0.22.2 -qqq
!pip install scipy~=1.1.0 -qqq
!pip install stable-baselines~=2.10.1 -qqq
!pip install tensorflow~=1.15.0 -qqq
!pip install torch>=1.6.0 -qqq
!pip install tqdm~=4.32.1 -qqq


     |████████████████████████████████| 2.5 MB 4.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.3-cp37-cp37m-linux_x86_64.whl size=2185307 sha256=704cfb823908446a273628a311115860409556da2caf4db978c289e0e9973212
  Stored in directory: /root/.cache/pip/wheels/7a/07/14/6a0c63fa2c6e473c6edc40985b7d89f05c61ff25ee7f0ad9ac
Successfully built mpi4py
     |████████████████████████████████| 448 kB 8.1 MB/s 
     |████████████████████████████████| 1.3 MB 7.4 MB/s 
     |████████████████████████████████| 1.6 MB 9.9 MB/s 
     |████████████████████████████████| 20.1 MB 1.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow 2.8.0 require

#config

In [ ]:

config = {
    "ENV": "CartPole-v1",
    "ALG": "BCIRMStudent_Apr17",
    "NUM_TRAJS_GIVEN": 20, #
    "NUM_TRAINING_ENVS": 2,
    "NOISE_DIM": 4,
    "REP_SIZE": 16,
    "TRAJ_SHIFT": 20, # 20,
    "SAMPLING_RATE": 5,
    "NUM_STEPS_TRAIN": 10000,
    "NUM_TRAJS_VALID": 100,
    "NUM_REPETITIONS": 15,
    "BATCH_SIZE": 64,
    "MLP_WIDTHS": 64,
    "ADAM_ALPHA": 1e-3,
    "SGLD_BUFFER_SIZE": 10000,
    "SGLD_LEARN_RATE": 0.01,
    "SGLD_NOISE_COEF": 0.01,
    "SGLD_NUM_STEPS": 100,
    "SGLD_REINIT_FREQ": 0.05,
    "NUM_STEPS_TRAIN_ENERGY_MODEL": 1000,
    #"NUM_STEPS_TRAIN_VAE_MODEL": 20000,
    'TRIAL': 0
}


#config['ENV'] = "LunarLander-v2"
config['ENV'] = "CartPole-v1"

#config['METHOD'] = "BCIRM"
config['METHOD'] = "CCIL"




if config['ENV'] == "CartPole-v1":
    config["REP_SIZE"] = 16



if config['METHOD'] == 'BCIRM':
    config['l2_regularizer_weight'] = 0.001
    config['penalty_weight'] = 10000
    config['penalty_anneal_iters'] = 2500
elif config['METHOD'] == "iVAE_IRM":
    config["NUM_STEPS_TRAIN_VAE_MODEL"] = 10000
    config['PHASE2_SAMPLES'] = 50000
    #config['LATENT_DIM'] = config["REP_SIZE"]
elif config['METHOD'] == "CCIL":
    #config['LATENT_DIM'] = config["REP_SIZE"]
    config['MASK_PROB'] = 0.3
    #config['BATCH_SIZE'] = 5000


#testing/il

In [ ]:
import argparse
import os
import pickle

import gym
import numpy as np
import pandas as pd
import yaml
import numpy as np

from testing.paths import get_model_path, get_trajs_path  # pylint: disable=reimported

from contrib.energy_model import EnergyModel
from contrib.env_wrapper import EnvWrapper, get_test_mult_factors
from network import EnvDiscriminator
from network import FeaturesDecoder
from network import FeaturesEncoder
from network import MineNetwork 
from network import ObservationsDecoder
from network import StudentNetwork


from student import ICILStudent, BCStudent, BCIRMStudent, iVAE_IRMStudent, CCILStudent 
from testing.train_utils import fill_buffer, make_agent, save_results
from vae.ivae_wrapper import VAE_wrapper
  


/usr/local/lib/python3.7/dist-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


# make student

In [ ]:


# pylint: disable=redefined-outer-name
def make_student(run_seed, config):
    env = gym.make(config["ENV"])
    trajs_path = get_trajs_path(config["ENV"], "student_" + config["ALG"], env_id="student", run_seed=run_seed)
    model_path = get_model_path(config["ENV"], "student_" + config["ALG"], run_seed=run_seed)

    state_dim = env.observation_space.shape[0] + config["NOISE_DIM"]
    action_dim = env.action_space.n
    num_training_envs = config["NUM_TRAINING_ENVS"]

    # run_seed = run_seed
    batch_size = config["BATCH_SIZE"]
    teacher = make_agent(config["ENV"], config["EXPERT_ALG"], config["NUM_TRAINING_ENVS"])
    teacher.load_pretrained()

    buffer = fill_buffer(
        trajs_path=teacher.trajs_paths,
        batch_size=batch_size,
        run_seed=run_seed,
        traj_shift=config["TRAJ_SHIFT"],
        buffer_size_in_trajs=config["NUM_TRAJS_GIVEN"],
        sampling_rate=config["SAMPLING_RATE"],
        strictly_batch_data = False
    )

    if buffer.total_size < batch_size:
        batch_size = buffer.total_size



    ##########################      COMMON      ##########################

    print("state_dim", state_dim)

    causal_features_encoder = FeaturesEncoder(
        input_size=state_dim, representation_size=config["REP_SIZE"], width=config["MLP_WIDTHS"]
    )

    policy_network = StudentNetwork(in_dim=config["REP_SIZE"], out_dim=action_dim, width=config["MLP_WIDTHS"])

    #print("config method = ", config['METHOD'])


    ##########################       BC       #######################

    if config['METHOD'] == 'BC':

        return BCStudent(
            env=env,
            trajs_paths=trajs_path,
            model_path=model_path,
            num_training_envs=num_training_envs,
            teacher=teacher,
            causal_features_encoder=causal_features_encoder,
            policy_network=policy_network,
            buffer=buffer,
            adam_alpha=config["ADAM_ALPHA"],
            config = config
        )


    ##########################       BC IRM       #######################


    elif config['METHOD'] == 'BCIRM':

        return BCIRMStudent(
            env=env,
            trajs_paths=trajs_path,
            model_path=model_path,
            num_training_envs=num_training_envs,
            teacher=teacher,
            causal_features_encoder=causal_features_encoder,
            policy_network=policy_network,
            buffer=buffer,
            adam_alpha=config["ADAM_ALPHA"],
            config = config
        )



    ##########################       CCIL       #######################


    elif config['METHOD'] == 'CCIL':
        
        causal_features_encoder = FeaturesEncoder(input_size=state_dim, representation_size=config["REP_SIZE"], width=config["MLP_WIDTHS"])

        return CCILStudent(
            env=env,
            trajs_paths=trajs_path,
            model_path=model_path,
            num_training_envs=num_training_envs,
            teacher=teacher,
            causal_features_encoder=causal_features_encoder,
            policy_network=policy_network,
            buffer=buffer,
            adam_alpha=config["ADAM_ALPHA"],
            config = config
        )

    ##########################       iVAE IRM       #######################

    elif config['METHOD'] == 'iVAE_IRM':

        config['LATENT_DIM'] = state_dim 

        vae_wrapper =  VAE_wrapper(buffer, data_dim = state_dim, action_dim = action_dim, env_dim = config['NUM_TRAINING_ENVS'], latent_dim = config['LATENT_DIM'], use_e = True)
        vae_wrapper.train(num_updates=config["NUM_STEPS_TRAIN_VAE_MODEL"])
        #vae_wrapper.start_phase2(n_samples = config['PHASE2_SAMPLES'])
        vae_wrapper.pa_list = [0,1,2,3,4,5,6,7]
        

        causal_features_encoder = FeaturesEncoder(
            input_size=config["LATENT_DIM"], representation_size=config["REP_SIZE"], width=config["MLP_WIDTHS"] )

        policy_network = StudentNetwork(in_dim=config["REP_SIZE"], out_dim=action_dim, width=config["MLP_WIDTHS"])

        phase3_obs_to_latent_encoder = FeaturesEncoder(
            input_size=state_dim, representation_size=config["LATENT_DIM"], width=config["MLP_WIDTHS"] )
        
        #causal_features_encoder = FeaturesEncoder(
        #            input_size=state_dim, representation_size = len(vae_wrapper.pa_list), width=config["MLP_WIDTHS"])

        #policy_network = StudentNetwork(in_dim = config['LATENT_DIM'], out_dim=action_dim, width=config["MLP_WIDTHS"])

        
        return iVAE_IRMStudent(
            env=env,
            vae_wrapper = vae_wrapper,
            trajs_paths=trajs_path,
            model_path=model_path,
            num_training_envs=num_training_envs,
            teacher=teacher,
            phase3_obs_to_latent_encoder = phase3_obs_to_latent_encoder,
            causal_features_encoder=causal_features_encoder,
            policy_network=policy_network,
            buffer=buffer,
            adam_alpha=config["ADAM_ALPHA"],
            config = config
        )
        


    ##########################       ICIL        #######################

    elif config['METHOD'] == 'ICIL':
        energy_model = EnergyModel(
            in_dim=state_dim,
            width=config["MLP_WIDTHS"],
            batch_size=batch_size,
            adam_alpha=config["ADAM_ALPHA"],
            buffer=buffer,
            sgld_buffer_size=config["SGLD_BUFFER_SIZE"],
            sgld_learn_rate=config["SGLD_LEARN_RATE"],
            sgld_noise_coef=config["SGLD_NOISE_COEF"],
            sgld_num_steps=config["SGLD_NUM_STEPS"],
            sgld_reinit_freq=config["SGLD_REINIT_FREQ"],
        )
        energy_model.train(num_updates=config["NUM_STEPS_TRAIN_ENERGY_MODEL"])

        causal_features_decoder = FeaturesDecoder(action_size=action_dim, representation_size=config["REP_SIZE"], width=config["MLP_WIDTHS"])

        observations_decoder = ObservationsDecoder(representation_size=config["REP_SIZE"], out_size=state_dim, width=config["MLP_WIDTHS"] )

        env_discriminator = EnvDiscriminator(representation_size=config["REP_SIZE"], num_envs=config["NUM_TRAINING_ENVS"], width=config["MLP_WIDTHS"])

        noise_features_encoders = [FeaturesEncoder(input_size=state_dim, representation_size=config["REP_SIZE"], width=config["MLP_WIDTHS"])  
            for i in range(num_training_envs)]
        
        noise_features_decoders = [FeaturesDecoder(action_size=action_dim, representation_size=config["REP_SIZE"], width=config["MLP_WIDTHS"])
            for i in range(num_training_envs)]

        mine_network = MineNetwork(x_dim=config["REP_SIZE"], z_dim=config["REP_SIZE"], width=config["MLP_WIDTHS"])

        return ICILStudent(
            env=env,
            trajs_paths=trajs_path,
            model_path=model_path,
            num_training_envs=num_training_envs,
            teacher=teacher,
            causal_features_encoder=causal_features_encoder,
            noise_features_encoders=noise_features_encoders,
            causal_features_decoder=causal_features_decoder,
            noise_features_decoders=noise_features_decoders,
            observations_decoder=observations_decoder,
            env_discriminator=env_discriminator,
            policy_network=policy_network,
            energy_model=energy_model,
            mine_network=mine_network,
            buffer=buffer,
            adam_alpha=config["ADAM_ALPHA"],
            config = config
        )


def init_arg():
    parser = argparse.ArgumentParser()
    parser.add_argument("--env_name", default="CartPole-v1")
    parser.add_argument("--num_trajectories", default=20, type=int)
    parser.add_argument("--trial", default=0, type=int)
    return parser.parse_args()


#10 Trails -- CCIL -- cartpole

In [ ]:
config['MASK_PROB'] = 0.5
config['NUM_STEPS_TRAIN'] = 100000
config['ADAM_ALPHA'] = 0.001
config['NUM_REPETITIONS'] = 10
config['ENV'] = 'CartPole-v1'

In [ ]:
config['METHOD'] = "CCIL"

for traj_num in [1, 2, 5, 10, 20, 30, 40, 50]:
    config["NUM_TRAJS_GIVEN"] = traj_num
    config["TRAJ_SHIFT"] = traj_num

    config['ALG'] = "FINAL_Apr24_CCILStudent_origindata_multiply_trajnum" + str(traj_num)


    ###############.  settings   ###############
    #config['ALG'] = "BCIRMStudent_Apr19_replicatedata"
    #config['METHOD'] = "BCIRM"
    #config['METHOD'] = "ICIL"
    #config["NUM_TRAJS_GIVEN"] = 50
    #config["TRAJ_SHIFT"] = 50
    #config['ENV'] == "CartPole-v1"
    ###############.  settings   ###############

    all_results_trail = []

    for trail in range(1): 
        config['TRIAL'] = trail 


        ###############.  start a trail   ###############

        config["EXPERT_ALG"] = yaml.load(open("testing/config.yml"), Loader=yaml.FullLoader)[config["ENV"]]
        print("Config: %s" % config)

        TRIAL = config["TRIAL"] #args.trial
        print("Trial number %s" % TRIAL)

        results_dir_base = "testing/results/"
        results_dir = os.path.join(results_dir_base, config["ENV"], str(config["NUM_TRAJS_GIVEN"]), config["ALG"])

        if not os.path.exists(results_dir):
            os.makedirs(results_dir)

        config_file = "trial_" + str(TRIAL) + "_" + "config.pkl"

        results_file_name = "trial_" + str(TRIAL) + "_" + "results.csv"
        results_file_path = os.path.join(results_dir, results_file_name)

        if os.path.exists(os.path.join(results_dir, config_file)):
            raise NameError("CONFIG file already exists %s. Choose a different trial number." % config_file)
        pickle.dump(config, open(os.path.join(results_dir, config_file), "wb"))




        ###############.  10 runs for each trail   ###############

        print("config method = ", config['METHOD'])
        print("config env = ", config['ENV'])

        for run_seed in range(config["NUM_REPETITIONS"]):
            print("Run %s out of %s" % (run_seed + 1, config["NUM_REPETITIONS"]))
            student = make_student(run_seed, config)
            student.train(num_updates=config["NUM_STEPS_TRAIN"])

            env_wrapper_out_of_sample = EnvWrapper(
                env=gym.make(config["ENV"]), mult_factor=get_test_mult_factors(config['NOISE_DIM'] - 1), idx=3, seed=1
            )

            env_wrapper_out_of_sample.noise = 0

            action_match, return_mean, return_std = student.test(
                num_episodes=config["NUM_TRAJS_VALID"], env_wrapper=env_wrapper_out_of_sample
            )

            result = (action_match, return_mean, return_std)
            print("###############    Reward for test environment for run %s: %s.   ###############\n\n" % (run_seed + 1, return_mean))
            save_results(results_file_path, run_seed, action_match, return_mean, return_std)

        results_trial = pd.read_csv(
            "testing/results/"
            + config["ENV"]
            + "/"
            + str(config["NUM_TRAJS_GIVEN"])
            + "/"
            + config["ALG"]
            + "/trial_"
            + str(TRIAL)
            + "_results.csv",
            header=None,
        )

        print("Average reward for 10 repetitions: %s" % np.mean(results_trial[2].values))

        all_results_trail.append(np.mean(results_trial[2].values))
    print("ALL RESULTS TRAIL:" , all_results_trail)




Config: {'ENV': 'CartPole-v1', 'ALG': 'FINAL_Apr24_CCILStudent_origindata_multiply_trajnum1', 'NUM_TRAJS_GIVEN': 1, 'NUM_TRAINING_ENVS': 2, 'NOISE_DIM': 4, 'REP_SIZE': 16, 'TRAJ_SHIFT': 1, 'SAMPLING_RATE': 5, 'NUM_STEPS_TRAIN': 100000, 'NUM_TRAJS_VALID': 100, 'NUM_REPETITIONS': 10, 'BATCH_SIZE': 64, 'MLP_WIDTHS': 64, 'ADAM_ALPHA': 0.001, 'SGLD_BUFFER_SIZE': 10000, 'SGLD_LEARN_RATE': 0.01, 'SGLD_NOISE_COEF': 0.01, 'SGLD_NUM_STEPS': 100, 'SGLD_REINIT_FREQ': 0.05, 'NUM_STEPS_TRAIN_ENERGY_MODEL': 1000, 'TRIAL': 0, 'METHOD': 'CCIL', 'MASK_PROB': 0.5, 'EXPERT_ALG': 'dqn'}
Trial number 0
config method =  CCIL
config env =  CartPole-v1
Run 1 out of 10
state_dim 8
epoch 49000/100000, policy loss 0.2493583858013153 	

#10 Trails -- CCIL -- acrobot * * *

In [ ]:
config['MASK_PROB'] = 0.5
config['NUM_STEPS_TRAIN'] = 100000
config['ADAM_ALPHA'] = 0.001
config['NUM_REPETITIONS'] = 10
config['ENV'] = "Acrobot-v1"

In [9]:
config['METHOD'] = "CCIL"

for traj_num in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20]:
    config["NUM_TRAJS_GIVEN"] = traj_num
    config["TRAJ_SHIFT"] = traj_num

    config['ALG'] = "FINAL_Apr24_CCILStudent_origindata_multiply_trajnum" + str(traj_num)


    ###############.  settings   ###############
    #config['ALG'] = "BCIRMStudent_Apr19_replicatedata"
    #config['METHOD'] = "BCIRM"
    #config['METHOD'] = "ICIL"
    #config["NUM_TRAJS_GIVEN"] = 50
    #config["TRAJ_SHIFT"] = 50
    #config['ENV'] == "CartPole-v1"
    ###############.  settings   ###############

    all_results_trail = []

    for trail in range(1): 
        config['TRIAL'] = trail 


        ###############.  start a trail   ###############

        config["EXPERT_ALG"] = yaml.load(open("testing/config.yml"), Loader=yaml.FullLoader)[config["ENV"]]
        print("Config: %s" % config)

        TRIAL = config["TRIAL"] #args.trial
        print("Trial number %s" % TRIAL)

        results_dir_base = "testing/results/"
        results_dir = os.path.join(results_dir_base, config["ENV"], str(config["NUM_TRAJS_GIVEN"]), config["ALG"])

        if not os.path.exists(results_dir):
            os.makedirs(results_dir)

        config_file = "trial_" + str(TRIAL) + "_" + "config.pkl"

        results_file_name = "trial_" + str(TRIAL) + "_" + "results.csv"
        results_file_path = os.path.join(results_dir, results_file_name)

        if os.path.exists(os.path.join(results_dir, config_file)):
            raise NameError("CONFIG file already exists %s. Choose a different trial number." % config_file)
        pickle.dump(config, open(os.path.join(results_dir, config_file), "wb"))




        ###############.  10 runs for each trail   ###############

        print("config method = ", config['METHOD'])
        print("config env = ", config['ENV'])

        for run_seed in range(config["NUM_REPETITIONS"]):
            print("Run %s out of %s" % (run_seed + 1, config["NUM_REPETITIONS"]))
            student = make_student(run_seed, config)
            student.train(num_updates=config["NUM_STEPS_TRAIN"])

            env_wrapper_out_of_sample = EnvWrapper(
                env=gym.make(config["ENV"]), mult_factor=get_test_mult_factors(config['NOISE_DIM'] - 1), idx=3, seed=1
            )

            env_wrapper_out_of_sample.noise = 0

            action_match, return_mean, return_std = student.test(
                num_episodes=config["NUM_TRAJS_VALID"], env_wrapper=env_wrapper_out_of_sample
            )

            result = (action_match, return_mean, return_std)
            print("###############    Reward for test environment for run %s: %s.   ###############\n\n" % (run_seed + 1, return_mean))
            save_results(results_file_path, run_seed, action_match, return_mean, return_std)

        results_trial = pd.read_csv(
            "testing/results/"
            + config["ENV"]
            + "/"
            + str(config["NUM_TRAJS_GIVEN"])
            + "/"
            + config["ALG"]
            + "/trial_"
            + str(TRIAL)
            + "_results.csv",
            header=None,
        )

        print("Average reward for 10 repetitions: %s" % np.mean(results_trial[2].values))

        all_results_trail.append(np.mean(results_trial[2].values))
    print("ALL RESULTS TRAIL:" , all_results_trail)




Config: {'ENV': 'Acrobot-v1', 'ALG': 'FINAL_Apr24_CCILStudent_origindata_multiply_trajnum1', 'NUM_TRAJS_GIVEN': 1, 'NUM_TRAINING_ENVS': 2, 'NOISE_DIM': 4, 'REP_SIZE': 16, 'TRAJ_SHIFT': 1, 'SAMPLING_RATE': 5, 'NUM_STEPS_TRAIN': 100000, 'NUM_TRAJS_VALID': 100, 'NUM_REPETITIONS': 10, 'BATCH_SIZE': 64, 'MLP_WIDTHS': 64, 'ADAM_ALPHA': 0.001, 'SGLD_BUFFER_SIZE': 10000, 'SGLD_LEARN_RATE': 0.01, 'SGLD_NOISE_COEF': 0.01, 'SGLD_NUM_STEPS': 100, 'SGLD_REINIT_FREQ': 0.05, 'NUM_STEPS_TRAIN_ENERGY_MODEL': 1000, 'TRIAL': 0, 'METHOD': 'CCIL', 'MASK_PROB': 0.5, 'EXPERT_ALG': 'dqn'}
Trial number 0
config method =  CCIL
config env =  Acrobot-v1
Run 1 out of 10


/content/drive/.shortcut-targets-by-id/15_BhZyJSvQWMTBbzA9iHkhBymeY5HxXY/Invariant-Causal-Imitation-Learning-main/testing/train_utils.py:106: UserWarning: Buffer smaller than batch size
  warnings.warn("Buffer smaller than batch size")


state_dim 10
epoch 99000/100000, policy loss 0.050731316208839417 	epoch 0/100 return: -71.0
epoch 10/100 return: -81.0
epoch 20/100 return: -83.0
epoch 30/100 return: -101.0
epoch 40/100 return: -88.0
epoch 50/100 return: -77.0
epoch 60/100 return: -85.0
epoch 70/100 return: -84.0
epoch 80/100 return: -71.0
epoch 90/100 return: -70.0
###############    Reward for test environment for run 1: -86.02.   ###############


Run 2 out of 10


/content/drive/.shortcut-targets-by-id/15_BhZyJSvQWMTBbzA9iHkhBymeY5HxXY/Invariant-Causal-Imitation-Learning-main/testing/train_utils.py:106: UserWarning: Buffer smaller than batch size
  warnings.warn("Buffer smaller than batch size")


state_dim 10
epoch 99000/100000, policy loss 0.00038851771387271583 	epoch 0/100 return: -72.0
epoch 10/100 return: -78.0
epoch 20/100 return: -126.0
epoch 30/100 return: -98.0
epoch 40/100 return: -73.0
epoch 50/100 return: -85.0
epoch 60/100 return: -79.0
epoch 70/100 return: -76.0
epoch 80/100 return: -72.0
epoch 90/100 return: -89.0
###############    Reward for test environment for run 2: -88.19.   ###############


Run 3 out of 10


...

state_dim 10
epoch 99000/100000, policy loss 0.11353497952222824 	epoch 0/100 return: -66.0
epoch 10/100 return: -123.0
epoch 20/100 return: -83.0
epoch 30/100 return: -87.0
epoch 40/100 return: -65.0
epoch 50/100 return: -78.0
epoch 60/100 return: -81.0
epoch 70/100 return: -85.0
epoch 80/100 return: -73.0
epoch 90/100 return: -82.0
###############    Reward for test environment for run 7: -84.06.   ###############


Run 8 out of 10
state_dim 10
epoch 99000/100000, policy loss 0.05620826780796051 	epoch 0/100 return: -65.0
epoch 10/100 return: -84.0
epoch 20/100 return: -93.0
epoch 30/100 return: -73.0
epoch 40/100 return: -81.0
epoch 50/100 return: -78.0
epoch 60/100 return: -70.0
epoch 70/100 return: -101.0
epoch 80/100 return: -85.0
epoch 90/100 return: -78.0
###############    Reward for test environment for run 8: -82.02.   ###############


Run 9 out of 10


/content/drive/.shortcut-targets-by-id/15_BhZyJSvQWMTBbzA9iHkhBymeY5HxXY/Invariant-Causal-Imitation-Learning-main/testing/train_utils.py:106: UserWarning: Buffer smaller than batch size
  warnings.warn("Buffer smaller than batch size")


state_dim 10
epoch 99000/100000, policy loss 0.001993665937334299 	epoch 0/100 return: -72.0
epoch 10/100 return: -78.0
epoch 20/100 return: -74.0
epoch 30/100 return: -98.0
epoch 40/100 return: -100.0
epoch 50/100 return: -77.0
epoch 60/100 return: -73.0
epoch 70/100 return: -72.0
epoch 80/100 return: -77.0
epoch 90/100 return: -90.0
###############    Reward for test environment for run 9: -84.85.   ###############


Run 10 out of 10


/content/drive/.shortcut-targets-by-id/15_BhZyJSvQWMTBbzA9iHkhBymeY5HxXY/Invariant-Causal-Imitation-Learning-main/testing/train_utils.py:106: UserWarning: Buffer smaller than batch size
  warnings.warn("Buffer smaller than batch size")


state_dim 10
epoch 99000/100000, policy loss 0.04966830462217331 	epoch 0/100 return: -72.0
epoch 10/100 return: -78.0
epoch 20/100 return: -72.0
epoch 30/100 return: -89.0
epoch 40/100 return: -71.0
epoch 50/100 return: -141.0
epoch 60/100 return: -72.0
epoch 70/100 return: -69.0
epoch 80/100 return: -77.0
epoch 90/100 return: -108.0
###############    Reward for test environment for run 10: -85.08.   ###############


Average reward for 10 repetitions: -84.61200000000001
ALL RESULTS TRAIL: [-84.61200000000001]
Config: {'ENV': 'Acrobot-v1', 'ALG': 'FINAL_Apr24_CCILStudent_origindata_multiply_trajnum3', 'NUM_TRAJS_GIVEN': 3, 'NUM_TRAINING_ENVS': 2, 'NOISE_DIM': 4, 'REP_SIZE': 16, 'TRAJ_SHIFT': 3, 'SAMPLING_RATE': 5, 'NUM_STEPS_TRAIN': 100000, 'NUM_TRAJS_VALID': 100, 'NUM_REPETITIONS': 10, 'BATCH_SIZE': 64, 'MLP_WIDTHS': 64, 'ADAM_ALPHA': 0.001, 'SGLD_BUFFER_SIZE': 10000, 'SGLD_LEARN_RATE': 0.01, 'SGLD_NOISE_COEF': 0.01, 'SGLD_NUM_STEPS': 100, 'SGLD_REINIT_FREQ': 0.05, 'NUM_STEPS_TRAIN_

#10 Trails -- CCIL -- lunarlander

In [ ]:
config['MASK_PROB'] = 0.5
config['NUM_STEPS_TRAIN'] = 100000
config['ADAM_ALPHA'] = 0.001
config['NUM_REPETITIONS'] = 10
config['ENV'] = 'LunarLander-v2'

In [ ]:
config['METHOD'] = "CCIL"

for traj_num in [2, 4, 8, 16, 32, 64, 128]:
    config["NUM_TRAJS_GIVEN"] = traj_num
    config["TRAJ_SHIFT"] = traj_num

    config['ALG'] = "FINAL_Apr24_CCILStudent_origindata_multiply_trajnum" + str(traj_num)


    ###############.  settings   ###############
    #config['ALG'] = "BCIRMStudent_Apr19_replicatedata"
    #config['METHOD'] = "BCIRM"
    #config['METHOD'] = "ICIL"
    #config["NUM_TRAJS_GIVEN"] = 50
    #config["TRAJ_SHIFT"] = 50
    #config['ENV'] == "CartPole-v1"
    ###############.  settings   ###############

    all_results_trail = []

    for trail in range(1): 
        config['TRIAL'] = trail 


        ###############.  start a trail   ###############

        config["EXPERT_ALG"] = yaml.load(open("testing/config.yml"), Loader=yaml.FullLoader)[config["ENV"]]
        print("Config: %s" % config)

        TRIAL = config["TRIAL"] #args.trial
        print("Trial number %s" % TRIAL)

        results_dir_base = "testing/results/"
        results_dir = os.path.join(results_dir_base, config["ENV"], str(config["NUM_TRAJS_GIVEN"]), config["ALG"])

        if not os.path.exists(results_dir):
            os.makedirs(results_dir)

        config_file = "trial_" + str(TRIAL) + "_" + "config.pkl"

        results_file_name = "trial_" + str(TRIAL) + "_" + "results.csv"
        results_file_path = os.path.join(results_dir, results_file_name)

        if os.path.exists(os.path.join(results_dir, config_file)):
            raise NameError("CONFIG file already exists %s. Choose a different trial number." % config_file)
        pickle.dump(config, open(os.path.join(results_dir, config_file), "wb"))




        ###############.  10 runs for each trail   ###############

        print("config method = ", config['METHOD'])
        print("config env = ", config['ENV'])

        for run_seed in range(config["NUM_REPETITIONS"]):
            print("Run %s out of %s" % (run_seed + 1, config["NUM_REPETITIONS"]))
            student = make_student(run_seed, config)
            student.train(num_updates=config["NUM_STEPS_TRAIN"])

            env_wrapper_out_of_sample = EnvWrapper(
                env=gym.make(config["ENV"]), mult_factor=get_test_mult_factors(config['NOISE_DIM'] - 1), idx=3, seed=1
            )

            env_wrapper_out_of_sample.noise = 0

            action_match, return_mean, return_std = student.test(
                num_episodes=config["NUM_TRAJS_VALID"], env_wrapper=env_wrapper_out_of_sample
            )

            result = (action_match, return_mean, return_std)
            print("###############    Reward for test environment for run %s: %s.   ###############\n\n" % (run_seed + 1, return_mean))
            save_results(results_file_path, run_seed, action_match, return_mean, return_std)

        results_trial = pd.read_csv(
            "testing/results/"
            + config["ENV"]
            + "/"
            + str(config["NUM_TRAJS_GIVEN"])
            + "/"
            + config["ALG"]
            + "/trial_"
            + str(TRIAL)
            + "_results.csv",
            header=None,
        )

        print("Average reward for 10 repetitions: %s" % np.mean(results_trial[2].values))

        all_results_trail.append(np.mean(results_trial[2].values))
    print("ALL RESULTS TRAIL:" , all_results_trail)




Config: {'ENV': 'CartPole-v1', 'ALG': 'FINAL_Apr24_CCILStudent_origindata_multiply_trajnum1', 'NUM_TRAJS_GIVEN': 1, 'NUM_TRAINING_ENVS': 2, 'NOISE_DIM': 4, 'REP_SIZE': 16, 'TRAJ_SHIFT': 1, 'SAMPLING_RATE': 5, 'NUM_STEPS_TRAIN': 100000, 'NUM_TRAJS_VALID': 100, 'NUM_REPETITIONS': 10, 'BATCH_SIZE': 64, 'MLP_WIDTHS': 64, 'ADAM_ALPHA': 0.001, 'SGLD_BUFFER_SIZE': 10000, 'SGLD_LEARN_RATE': 0.01, 'SGLD_NOISE_COEF': 0.01, 'SGLD_NUM_STEPS': 100, 'SGLD_REINIT_FREQ': 0.05, 'NUM_STEPS_TRAIN_ENERGY_MODEL': 1000, 'TRIAL': 0, 'METHOD': 'CCIL', 'MASK_PROB': 0.5, 'EXPERT_ALG': 'dqn'}
Trial number 0
config method =  CCIL
config env =  CartPole-v1
Run 1 out of 10
state_dim 8
epoch 2000/100000, policy loss 0.5670180916786194 	

#trash

In [ ]:

def _compute_loss(samples):
    state = torch.FloatTensor(samples["state"]).to(self.device)
    action = torch.LongTensor(samples["action"]).to(self.device)
    #next_state = torch.FloatTensor(samples["next_state"]).to(self.device)
    #env_ids = torch.LongTensor(samples["env"]).to(self.device)
    
            
    prob = torch.ones(state.size()) * (1 - self.mask_prob)
    
    mask = torch.bernoulli(prob).to(self.device)
    state_concat = torch.cat([state * mask, mask], dim=1)

    causal_rep = self.causal_features_encoder(state_concat)  # need this encoder: S -> rep

    # 1. Policy loss
    qvalues = self.policy_network(causal_rep) # need this encoder:  rep -> A
    ce_loss = nn.CrossEntropyLoss()(qvalues, action)

    return ce_loss, mask


In [ ]:
import torch.nn as nn

num_masks = 100000
best_mask = None
best_loss = np.inf

self = student 


for i in range(num_masks): 
    sample = self.buffer.sample()
    ce_loss, curr_mask = _compute_loss(sample)
    if ce_loss < best_loss:
        best_mask = curr_mask
        print(ce_loss)
        best_loss = ce_loss



tensor(0.4874, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4773, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4761, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4751, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4727, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
state = sample['state'] 
prob = torch.ones(state.shape[1]) * (1 - self.mask_prob)
mask = torch.bernoulli(prob).to(self.device)
mask = torch.tile(mask, (state.shape[0],1))
state_concat = torch.cat([state * mask, mask], dim=1)
causal_rep = self.causal_features_encoder(state_concat)  # need this encoder: S -> rep

# 1. Policy loss
qvalues = self.policy_network(causal_rep) # need this encoder:  rep -> A
ce_loss = nn.CrossEntropyLoss()(qvalues, action)


tensor([[0., 0., 0.,  ..., 1., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 1.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 1.]], device='cuda:0')

In [ ]:
student.best_mask[0,:]

tensor([1., 0., 0., 0., 0., 1., 1., 1.], device='cuda:0')

In [ ]:
env_wrapper_out_of_sample = EnvWrapper(
    env=gym.make(config["ENV"]), mult_factor=get_test_mult_factors(config['NOISE_DIM'] - 1), idx=3, seed=1
)

env_wrapper_out_of_sample.noise = 0

action_match, return_mean, return_std = student.test(
    num_episodes=config["NUM_TRAJS_VALID"], env_wrapper=env_wrapper_out_of_sample
)

result = (action_match, return_mean, return_std)
result

epoch 0/100 return: 10.0
epoch 10/100 return: 10.0
epoch 20/100 return: 9.0
epoch 30/100 return: 9.0
epoch 40/100 return: 8.0
epoch 50/100 return: 9.0
epoch 60/100 return: 10.0
epoch 70/100 return: 9.0
epoch 80/100 return: 10.0
epoch 90/100 return: 10.0


(0.0, 9.36, 0.7282856582413249)

In [ ]:
student.buffer.total_size

498000

In [ ]:
config

{'ADAM_ALPHA': 0.001,
 'ALG': 'BCIRMStudent_Apr17',
 'BATCH_SIZE': 64,
 'ENV': 'CartPole-v1',
 'MASK_PROB': 0.5,
 'METHOD': 'CCIL',
 'MLP_WIDTHS': 64,
 'NOISE_DIM': 4,
 'NUM_REPETITIONS': 15,
 'NUM_STEPS_TRAIN': 100000,
 'NUM_STEPS_TRAIN_ENERGY_MODEL': 1000,
 'NUM_TRAINING_ENVS': 2,
 'NUM_TRAJS_GIVEN': 20,
 'NUM_TRAJS_VALID': 100,
 'REP_SIZE': 16,
 'SAMPLING_RATE': 5,
 'SGLD_BUFFER_SIZE': 10000,
 'SGLD_LEARN_RATE': 0.01,
 'SGLD_NOISE_COEF': 0.01,
 'SGLD_NUM_STEPS': 100,
 'SGLD_REINIT_FREQ': 0.05,
 'TRAJ_SHIFT': 20,
 'TRIAL': 0}